In [1]:
import h5py
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import ElasticNet, LinearRegression, Ridge, ElasticNetCV, LassoCV, Lasso
from sklearn.model_selection import GridSearchCV, KFold, train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler, RobustScaler, scale
from distfit import distfit
from sklearn.decomposition import PCA
from scipy.stats import boxcox 
import seaborn as sns 
from haversine import haversine

In [2]:
tracts_deserts = pd.read_csv('nyc_desert_tracts.csv', dtype={'GEOID':'str'})
tracts_deserts

,food_closest_travel_times,physical_closest_dist,transport_closest_dist,education_closest_travel_times,worship_closest_travel_times,GEOID
0,241.000061,0.000338,0.000338,228.306671,239.923798,36005000100
1,279.976196,0.282170,0.307877,60.904408,41.152637,36005000200
2,293.006378,0.273411,0.194327,66.519920,66.196022,36005000400
3,155.181366,0.247503,0.087596,20.076658,17.588924,36005001600
4,124.599907,0.048832,0.286151,108.333473,127.937584,36005001900
...,...,...,...,...,...,...
2163,40.582970,0.353592,0.364519,105.080574,91.388008,36085030302
2164,83.954140,0.232066,0.393496,85.947723,14.448265,36085031901
2165,137.402847,0.318557,0.173371,36.799999,82.008224,36085031902
2166,189.980087,0.411144,0.320570,89.377243,136.685715,36085032300


In [3]:
zcta_health = pd.read_csv("PLACES__ZCTA_Data__GIS_Friendly_Format___2021_release.csv", dtype={'ZCTA5':'str', 'GEOID':'str'})
zcta_health

,ZCTA5,TotalPopulation,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ARTHRITIS_CrudePrev,ARTHRITIS_Crude95CI,BINGE_CrudePrev,BINGE_Crude95CI,BPHIGH_CrudePrev,BPHIGH_Crude95CI,...,OBESITY_Crude95CI,PHLTH_CrudePrev,PHLTH_Crude95CI,SLEEP_CrudePrev,SLEEP_Crude95CI,STROKE_CrudePrev,STROKE_Crude95CI,TEETHLOST_CrudePrev,TEETHLOST_Crude95CI,Geolocation
0,08240,"2,371",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,35.8,"(32.4, 38.9)",NaN,NaN,NaN,NaN,POINT (-74.53173709 39.48730537)
1,08904,"13,982",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,34.2,"(33.4, 35.1)",NaN,NaN,7.7,"( 5.4, 10.7)",POINT (-74.4289405 40.49812351)
2,08065,"7,398",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,37.8,"(36.8, 38.8)",NaN,NaN,12.4,"(10.0, 15.1)",POINT (-75.035071 40.00536195)
3,28443,"17,512",12.4,"(11.2, 13.7)",28.6,"(27.8, 29.3)",16.8,"(16.5, 17.2)",33.1,"(32.3, 33.8)",...,"(30.2, 32.0)",11.4,"(10.8, 12.2)",33.3,"(32.5, 34.1)",3.2,"( 3.0, 3.4)",11.5,"( 9.3, 14.1)",POINT (-77.67400103 34.40735763)
4,03103,"36,476",17.7,"(16.8, 18.8)",23.5,"(23.1, 23.9)",18.6,"(18.4, 18.9)",29.4,"(29.0, 29.8)",...,"(35.2, 36.2)",15.0,"(14.4, 15.5)",39.6,"(39.0, 40.1)",3.0,"( 2.9, 3.1)",17.9,"(15.4, 20.5)",POINT (-71.44426868 42.95332718)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32404,90265,"18,116",6.4,"( 5.8, 7.0)",21.6,"(21.0, 22.2)",18.4,"(18.2, 18.7)",27.3,"(26.6, 27.9)",...,"(22.2, 23.3)",9.4,"( 8.9, 10.1)",30.6,"(29.9, 31.3)",2.6,"( 2.5, 2.8)",5.2,"( 4.0, 6.5)",POINT (-118.8177028 34.05791275)
32405,92567,"9,459",19.8,"(18.0, 21.6)",22.1,"(21.6, 22.6)",18.0,"(17.7, 18.4)",30.2,"(29.6, 30.7)",...,"(33.7, 34.9)",15.0,"(14.3, 15.7)",38.9,"(38.3, 39.5)",3.2,"( 3.0, 3.3)",16.5,"(13.9, 19.2)",POINT (-117.104794 33.81232975)
32406,98237,"4,025",12.3,"(10.3, 14.5)",30.9,"(29.5, 32.1)",14.8,"(14.3, 15.3)",32.3,"(30.8, 33.5)",...,"(30.7, 33.5)",16.0,"(14.4, 17.6)",33.8,"(32.7, 34.9)",4.0,"( 3.6, 4.5)",15.3,"(11.1, 19.8)",POINT (-121.6852387 48.59322161)
32407,87564,332,12.2,"(10.2, 14.4)",26.4,"(25.2, 27.5)",14.2,"(13.6, 14.8)",32.0,"(30.7, 33.1)",...,"(24.1, 26.3)",13.7,"(12.4, 15.0)",29.6,"(28.5, 30.8)",3.4,"( 3.1, 3.8)",12.6,"( 8.9, 17.2)",POINT (-105.6278913 36.60796943)


In [4]:
zip_to_tract = pd.read_csv("zcta_to_tract10.csv", dtype=str)
zip_to_tract = zip_to_tract[['GEOID', 'ZCTA5']]

In [5]:
tracts_zcta_deserts = pd.merge(tracts_deserts, zip_to_tract, on='GEOID')

In [6]:
tracts_zcta_deserts

,food_closest_travel_times,physical_closest_dist,transport_closest_dist,education_closest_travel_times,worship_closest_travel_times,GEOID,ZCTA5
0,241.000061,0.000338,0.000338,228.306671,239.923798,36005000100,11370
1,279.976196,0.282170,0.307877,60.904408,41.152637,36005000200,10473
2,293.006378,0.273411,0.194327,66.519920,66.196022,36005000400,10473
3,155.181366,0.247503,0.087596,20.076658,17.588924,36005001600,10473
4,124.599907,0.048832,0.286151,108.333473,127.937584,36005001900,10454
...,...,...,...,...,...,...,...
2860,40.582970,0.353592,0.364519,105.080574,91.388008,36085030302,10303
2861,40.582970,0.353592,0.364519,105.080574,91.388008,36085030302,10314
2862,83.954140,0.232066,0.393496,85.947723,14.448265,36085031901,10303
2863,137.402847,0.318557,0.173371,36.799999,82.008224,36085031902,10303


In [7]:
tracts_zcta_deserts = tracts_zcta_deserts.groupby('ZCTA5', as_index=False).median()
tracts_zcta_deserts

,ZCTA5,food_closest_travel_times,physical_closest_dist,transport_closest_dist,education_closest_travel_times,worship_closest_travel_times
0,10001,34.733772,0.167533,0.077006,59.555080,45.433716
1,10002,42.952351,0.076797,0.135394,25.395082,26.600000
2,10003,54.583931,0.154937,0.081738,20.200001,35.354568
3,10004,84.726910,0.081697,0.082260,31.929441,46.507748
4,10005,86.738968,0.144094,0.082453,28.493832,86.738968
...,...,...,...,...,...,...
209,11691,146.671173,0.207410,0.269644,57.347967,69.513790
210,11692,70.289391,0.156135,0.155561,41.700539,63.795357
211,11693,76.302540,0.120755,0.117169,53.875572,25.768712
212,11694,109.728714,0.268667,0.076101,71.228714,47.170155


In [8]:
zcta_health = pd.merge(zcta_health, tracts_zcta_deserts, on='ZCTA5')
#zcta_health.to_csv('zcta5_health_nyc.csv', index=False)

In [9]:
name_mapping = {'access2': 'Health insurance access', 'arthritis': 'Arthritis prevalence', 'binge': 'Binge drinking prevalence',
               'bphigh': 'High blood pressure prevalence', 'bpmed': 'Medium blood pressure prevalence', 'cancer': 'Cancer prevalence',
               'casthma': 'Asthma prevalence', 'cervical': 'Cervical cancer screenings', 'chd': 'Coronary heart disease prevalence',
               'checkup': 'Routine checkups', 'cholscreen': 'Cholesterol screenings', 'colon_screen': 'Colon cancer screenings',
               'copd': 'COPD prevalence', 'corem': 'Core men\'s health', 'corew': 'Core women\'s health', 'csmoking': 'Smoking prevalence',
               'dental': 'Dental checkups', 'depression': 'Depression prevalence', 'diabetes': 'Diabetes prevalence', 'ghlth': 'General poor health prevalence',
               'highchol': 'High cholesterol prevalence', 'kidney': 'Chronic kidney disease', 'lpa': 'No physical activity', 'mammouse': 'Mammograms',
               'mhlth': 'Poor mental health prevalence', 'obesity': 'Obesity prevalence', 'phlth': 'Poor physical health', 'sleep': 'Poor sleep prevalence',
               'stroke': 'Stroke prevalence', 'teethlost': 'Teeth loss prevalence'}

results_nan = pd.DataFrame({'Health condition': [np.nan for c in zcta_health.columns if c.endswith('CrudePrev')], 'Food': [np.nan for c in zcta_health.columns if c.endswith('CrudePrev')],
                       'Physical health': [np.nan for c in zcta_health.columns if c.endswith('CrudePrev')], 'Public transport': [np.nan for c in zcta_health.columns if c.endswith('CrudePrev')],
                        'Education': [np.nan for c in zcta_health.columns if c.endswith('CrudePrev')], 'Houses of worship': [np.nan for c in zcta_health.columns if c.endswith('CrudePrev')],
                       'RSquared': [np.nan for c in zcta_health.columns if c.endswith('CrudePrev')], 'MSE': [np.nan for c in zcta_health.columns if c.endswith('CrudePrev')]})

desert_measures = ['food_closest_travel_times', 'physical_closest_dist', 'transport_closest_dist', 'education_closest_travel_times', 'worship_closest_travel_times']

In [10]:
alphas = []
l1_ratios = []
i=0
results = results_nan.copy()

plt.figure(figsize=(20, 15))
plt.subplots_adjust(hspace=0.5)

for c in zcta_health.columns:
    if c.endswith('CrudePrev'):
        name = name_mapping[c[:-10].lower()]
        
        x = np.log(zcta_health[desert_measures].to_numpy(copy=True)+1)
        scaler = StandardScaler()
        xscale = scaler.fit_transform(x)
        y = zcta_health[c].to_numpy(copy=True)
        xscale = xscale[~np.isnan(y)]
        y = y[~np.isnan(y)]
        #create function to center data
        center_function = lambda x: x - x.mean()

        #apply function to original NumPy array
        data_centered = center_function(y)
        
        X_train, X_test, y_train, y_test = train_test_split(xscale, 
                                                    y, 
                                                    test_size=0.25, 
                                                    random_state=42)
        
        #l1 ratio is from suggested values in ElasticNetCV documentation
        enet_cv = ElasticNetCV(l1_ratio = [.1, .5, .7, .9, .95, .99, 1], 
                                     cv = 10, normalize=True).fit(X_train,y_train)
        
        alpha = enet_cv.alpha_
        l1 = enet_cv.l1_ratio_
        alphas.append(alpha)
        l1_ratios.append(l1)
        
        regr = ElasticNet(alpha=alpha, l1_ratio = l1, normalize=True)  # Could try others, or other parameters?
        regr.fit(X_train, y_train.reshape(-1, 1))
        
        predictions = regr.predict(X_test)
        y_train_pred = regr.predict(X_train)
        mse_test = mean_squared_error(y_test, predictions)
        
        results.iat[i, 0] = name
        results.iat[i, 1] = regr.coef_[1]
        results.iat[i, 2] = regr.coef_[3]
        results.iat[i, 3] = regr.coef_[4]
        results.iat[i, 4] = regr.coef_[0]
        results.iat[i, 5] = regr.coef_[2]
        results.iat[i, 6] = regr.score(X_test, y_test)
        results.iat[i, 7] = mse_test
            
        i += 1

results_round = results.round({'Food': 4, 'Physical health':4, 'Public transport':4, 'Education':4,
             'House of worship':4, 'RSquared':4})
results_round.sort_values(by='RSquared', ascending=False).reset_index(drop=True)

,Health condition,Food,Physical health,Public transport,Education,Houses of worship,RSquared,MSE
0,Binge drinking prevalence,-0.8366,-0.5179,0.4540,-0.5972,-7.987574e-01,0.4649,8.499719
1,High blood pressure prevalence,0.5905,0.4049,-1.4492,1.6101,8.421882e-01,0.3086,27.245988
2,Depression prevalence,-0.7810,-0.2437,0.3243,-0.0854,-2.988662e-01,0.2740,3.549193
3,Mammograms,-0.6524,-0.1711,-0.2151,0.2070,-8.200620e-01,0.2646,5.627455
4,Medium blood pressure prevalence,0.7330,0.7103,-0.7850,0.4568,1.223346e+00,0.2611,24.452145
5,Diabetes prevalence,0.3343,0.2368,-0.6705,0.6969,1.958623e-01,0.2134,7.893696
6,Arthritis prevalence,0.0000,0.0949,-0.1293,0.5712,8.478739e-01,0.2066,18.031270
7,Routine checkups,0.2468,0.1819,-0.1362,0.3616,3.554208e-01,0.1992,10.234266
8,COPD prevalence,0.0000,0.0501,-0.1041,0.3351,2.610245e-01,0.1914,2.090556
9,Cancer prevalence,-0.0000,0.0000,0.0000,0.0000,4.089647e-01,0.1625,4.674431


<Figure size 1440x1080 with 0 Axes>

In [11]:
alphas = []
l1_ratios = []
i=0
results = results_nan.copy()

plt.figure(figsize=(20, 15))
plt.subplots_adjust(hspace=0.5)

for c in zcta_health.columns:
    if c.endswith('CrudePrev'):
        name = name_mapping[c[:-10].lower()]
        
        x = zcta_health[desert_measures].to_numpy(copy=True)
        scaler = StandardScaler()
        xscale = scaler.fit_transform(x)
        y = zcta_health[c].to_numpy(copy=True)
        xscale = xscale[~np.isnan(y)]
        y = y[~np.isnan(y)]
        #create function to center data
        center_function = lambda x: x - x.mean()

        #apply function to original NumPy array
        y = center_function(y)
        
        X_train, X_test, y_train, y_test = train_test_split(xscale, 
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state=42)
        
        #l1 ratio is from suggested values in ElasticNetCV documentation
        lasso_cv = LassoCV(cv = 5, normalize=True).fit(X_train,y_train)
        
        alpha = lasso_cv.alpha_
        alphas.append(alpha)
        
        regr = Lasso(alpha=alpha, normalize=True)  # Could try others, or other parameters?
        regr.fit(X_train, y_train.reshape(-1, 1))
        
        predictions = regr.predict(X_test)
        y_train_pred = regr.predict(X_train)
        mse_test = mean_squared_error(y_test, predictions)
        
        results.iat[i, 0] = name
        results.iat[i, 1] = regr.coef_[1]
        results.iat[i, 2] = regr.coef_[3]
        results.iat[i, 3] = regr.coef_[4]
        results.iat[i, 4] = regr.coef_[0]
        results.iat[i, 5] = regr.coef_[2]
        results.iat[i, 6] = regr.score(X_test, y_test)
        results.iat[i, 7] = mse_test

        i += 1

results_round = results.round({'Food': 4, 'Physical health':4, 'Public transport':4, 'Education':4,
             'House of worship':4, 'RSquared':4})
results_round.sort_values(by='RSquared', ascending=False).reset_index(drop=True)

,Health condition,Food,Physical health,Public transport,Education,Houses of worship,RSquared,MSE
0,Binge drinking prevalence,-0.9571,-0.0000,0.6422,-0.5454,-1.047291,0.4325,9.065241
1,Depression prevalence,-0.9172,-0.1284,0.3427,-0.0000,-0.287828,0.3293,3.198292
2,Mammograms,-0.5323,0.0000,-0.0000,-0.0000,-0.860800,0.2577,6.126623
3,High blood pressure prevalence,0.8668,0.0000,-0.9310,0.9827,0.825605,0.2287,28.230161
4,Cancer prevalence,0.0000,0.0591,0.0520,0.0148,0.361179,0.1909,3.852047
5,Medium blood pressure prevalence,0.3447,0.0000,-0.0000,0.0000,0.763432,0.1639,24.472968
6,Arthritis prevalence,0.0000,0.0000,0.0000,0.3361,0.673475,0.1636,16.323167
7,Routine checkups,0.1928,0.0434,0.0000,0.2441,0.301009,0.1617,9.296448
8,Diabetes prevalence,0.4532,-0.0000,-0.7300,0.4691,0.261888,0.1589,8.520635
9,COPD prevalence,0.0148,-0.0709,-0.1585,0.2551,0.362204,0.1487,2.072656


<Figure size 1440x1080 with 0 Axes>

In [12]:
alphas = []
l1_ratios = []
i=0
results = results_nan.copy()

plt.figure(figsize=(20, 15))
plt.subplots_adjust(hspace=0.5)

for c in zcta_health.columns:
    if c.endswith('CrudePrev'):
        name = name_mapping[c[:-10].lower()]
        
        x = np.log(zcta_health[desert_measures].to_numpy(copy=True)+1)
        scaler = StandardScaler()
        xscale = scaler.fit_transform(x)
        y = zcta_health[c].to_numpy(copy=True)
        xscale = xscale[~np.isnan(y)]
        y = y[~np.isnan(y)]
        #create function to center data
        center_function = lambda x: x - x.mean()

        #apply function to original NumPy array
        y = center_function(y)
        
        X_train, X_test, y_train, y_test = train_test_split(xscale, 
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state=42)
        
        #l1 ratio is from suggested values in ElasticNetCV documentation
        lasso_cv = LassoCV(cv = 5, normalize=True).fit(X_train,y_train)
        
        alpha = lasso_cv.alpha_
        alphas.append(alpha)
        
        regr = Lasso(alpha=alpha, normalize=True)  # Could try others, or other parameters?
        regr.fit(X_train, y_train.reshape(-1, 1))
        
        y_test_pred = regr.predict(X_test)
        y_train_pred = regr.predict(X_train)
        mse_test = mean_squared_error(y_test, y_test_pred)
        
        results.iat[i, 0] = name
        results.iat[i, 1] = regr.coef_[1]
        results.iat[i, 2] = regr.coef_[3]
        results.iat[i, 3] = regr.coef_[4]
        results.iat[i, 4] = regr.coef_[0]
        results.iat[i, 5] = regr.coef_[2]
        results.iat[i, 6] = regr.score(X_test, y_test)
        results.iat[i, 7] = mse_test
            
        i += 1

results_round = results.round({'Food': 4, 'Physical health':4, 'Public transport':4, 'Education':4,
             'House of worship':4, 'RSquared':4}).sort_values(by='RSquared', ascending=False).reset_index(drop=True)
results_round

,Health condition,Food,Physical health,Public transport,Education,Houses of worship,RSquared,MSE
0,Binge drinking prevalence,-0.8094,-0.5561,0.5904,-0.6092,-9.445561e-01,0.4844,8.236136
1,Depression prevalence,-0.8583,-0.2393,0.2634,-0.0000,-2.516130e-01,0.3121,3.280416
2,High blood pressure prevalence,0.5247,0.4884,-1.5990,1.5823,8.720285e-01,0.3066,25.379300
3,Medium blood pressure prevalence,0.6471,0.7903,-1.0273,0.3573,1.483557e+00,0.2780,21.133689
4,Mammograms,-0.5113,-0.0000,-0.1016,0.0000,-8.546182e-01,0.2587,6.118442
5,Diabetes prevalence,0.2942,0.2508,-0.7404,0.8105,1.284392e-01,0.2294,7.806936
6,Routine checkups,0.2285,0.1927,-0.4039,0.5140,5.244275e-01,0.2181,8.670832
7,Arthritis prevalence,0.0000,0.0566,-0.0762,0.5101,7.711769e-01,0.1968,15.675332
8,COPD prevalence,0.0000,0.0587,-0.0764,0.3041,2.417723e-01,0.1962,1.957007
9,Obesity prevalence,0.4684,0.0000,-0.8376,2.1796,1.526586e-01,0.1751,26.408908


<Figure size 1440x1080 with 0 Axes>

In [16]:
alphas = []
l1_ratios = []
i=0
results = results_nan.copy()

plt.figure(figsize=(20, 15))
plt.subplots_adjust(hspace=0.5)

for c in zcta_health.columns:
    if c.endswith('CrudePrev'):
        name = name_mapping[c[:-10].lower()]
        
        x = np.log(zcta_health[desert_measures].to_numpy(copy=True)+1)
        scaler = StandardScaler()
        xscale = scaler.fit_transform(x)
        y = zcta_health[c].to_numpy(copy=True)
        xscale = xscale[~np.isnan(y)]
        y = y[~np.isnan(y)]
        #create function to center data
        center_function = lambda x: x - x.mean()

        #apply function to original NumPy array
        y = center_function(y)
        
        X_train, X_test, y_train, y_test = train_test_split(xscale, 
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state=42)
        
        #l1 ratio is from suggested values in ElasticNetCV documentation
        lasso_cv = LassoCV(cv = 5, normalize=True).fit(X_train,y_train)
        
        alpha = lasso_cv.alpha_
        alphas.append(alpha)
        
        regr = Lasso(alpha=alpha, normalize=True)  # Could try others, or other parameters?
        regr.fit(xscale, y.reshape(-1, 1))
        
        y_pred = regr.predict(xscale)
        mse = mean_squared_error(y, y_pred)
        
        results.iat[i, 0] = name
        results.iat[i, 1] = regr.coef_[1]
        results.iat[i, 2] = regr.coef_[3]
        results.iat[i, 3] = regr.coef_[4]
        results.iat[i, 4] = regr.coef_[0]
        results.iat[i, 5] = regr.coef_[2]
        results.iat[i, 6] = regr.score(xscale, y)
        results.iat[i, 7] = mse
            
        i += 1

results_round = results.round({'Food': 4, 'Physical health':4, 'Public transport':4, 'Education':4,
             'House of worship':4, 'RSquared':4}).sort_values(by='RSquared', ascending=False).reset_index(drop=True)
results_round.to_csv('nyc_lasso.csv')
results_round

,Health condition,Food,Physical health,Public transport,Education,Houses of worship,RSquared,MSE
0,Binge drinking prevalence,-1.1349,-0.3899,0.5885,-0.6997,-1.051212,0.3615,11.078724
1,Depression prevalence,-0.8528,-0.0796,0.3072,-0.0000,-0.449851,0.3243,3.246455
2,High blood pressure prevalence,0.9806,0.2135,-1.6886,1.9181,1.045811,0.2672,25.464614
3,Mammograms,-0.5191,-0.0000,-0.0293,0.0000,-0.894435,0.2641,5.751815
4,Medium blood pressure prevalence,0.9072,0.4962,-0.9712,0.4308,1.736297,0.2254,26.720726
5,Arthritis prevalence,0.0503,0.0000,-0.0000,0.5837,1.008939,0.2159,11.020792
6,Routine checkups,0.4839,0.0959,-0.3062,0.5605,0.693887,0.2099,6.775848
7,Obesity prevalence,0.2289,0.0000,-1.2453,2.4160,0.000000,0.2073,28.526744
8,Cancer prevalence,0.0000,0.0000,0.0291,0.0000,0.579350,0.1941,2.273303
9,Coronary heart disease prevalence,0.0685,0.0418,-0.1295,0.2652,0.364134,0.1887,1.646994


<Figure size 1440x1080 with 0 Axes>